# Grain Size Quantification and Mapping of Pebbles (Soloy et al. 2020)

This code allows the detection and measurement of the non-overlapping clasts visible on scales terrestrial photographs and on georeferenced UAV derived ortho-images, as described by Soloy et al. (2020).

The instance segmentation model named Mask R-CNN trained and use for this purpose was first developped by He et al. (2017).
The present code is based on the Matterport's implementation (https://github.com/matterport/Mask_RCNN)

- Soloy, A.; Turki, I.; Fournier, M.; Costa, S.; Peuziat, B.; Lecoq, N. A Deep Learning-Based Method for Quantifying and Mapping the Grain Size on Pebble Beaches. Remote Sens. 2020, 12, 3659.
- He, K.; Gkioxari, G.; Dollar, P.; Girshick, R. Mask R-CNN. In Proceedings of the 2017 IEEE International Conference on Computer Vision (ICCV), Venice, Italy, 22–29 October 2017; pp. 2980–2988.

## Configurations

In [ ]:
import os
import numpy as np
from os import listdir
from os.path import isfile, join
from functions import clasts_rasterize
from functions import clasts_detection
import pandas as pd

# Root directory of the project
RT_DIR = os.getcwd()

## Load Paths to data

In [ ]:
# Paths to directories of images to be measured
datadirpath = os.path.join(RT_DIR, "datasets")
terrestrialdirpath = os.path.join(datadirpath, "terrestrial")
UAVdirpath = os.path.join(datadirpath, "UAV")

## Run Detection & Measurements

### On terrestrial scaled photographs

In [ ]:
# List of image files to be analyzed 
# TO DO: Adapt the file extension to the appropriate type (jpg, tif, png, etc.) or erase the condition from "&"

filenames = [f for f in listdir(terrestrialdirpath) if isfile(join(terrestrialdirpath, f)) & (f[-4:]=='.jpg')]
numboffiles=np.shape(filenames)

for i in range(0, numboffiles[0]):
    print('['+str(i)+'] '+filenames[i])

In [ ]:
# Arguments list
mode = "terrestrial"
devicemode = "GPU"    #Chose between GPU and CPU depending on your configuration. (see https://github.com/matterport/Mask_RCNN)
devicenumber = 1      # Default value is 0 (i.e. first device in the list)

resolution = 0.001

#Detection & measurement operation
for i in range(0, numboffiles[0]):
    imgpath=os.path.join(terrestrialdirpath,filenames[i])
    clasts = clasts_detection.clasts_detect(mode = mode, resolution = resolution, imgpath = imgpath, plot = True, saveplot = True, saveresults = True)

# Detects and measure the clasts on a scaled image that can be either terrestrial (i.e. photograph of known resolution) or UAV derived (georeferenced ortho-image)
# mode: "terrestrial", "UAV"
# resolution: resolution of a terrestrial image (default = 0.001 m/pixel)
# imgpath: full path of the image/ortho-image
# metric_cropsize: Window size used for croping a UAV image into tiles (default = 1m)
# plot: (boolean) Display detections and histograms
# saveplot: (boolean) Save the detection and histogram figures
# saveresults: (boolean) Save the clast sizes into a CSV file
# devicemode: "gpu", "cpu" (default = "gpu") (see tesorflow-gpu documentation)
# devicenumber: id number of the device to be used (default = 0)

### On UAV-derived geotiff ortho-images

In [ ]:
# List of image raster files to be analyzed
# Files must be provided in a tif format using UTM or any other projected coordinate system allowing for metric measurements.
filenames = [f for f in listdir(UAVdirpath) if isfile(join(UAVdirpath, f)) & (f[-4:]=='.tif')]
numboffiles = np.shape(filenames)

for i in range(0, numboffiles[0]):
    print('[' + str(i) + '] ' + filenames[i])

In [ ]:
# Arguments list
mode = "UAV"
devicemode = "GPU"    # Chose between GPU and CPU depending on your configuration. (see https://github.com/matterport/Mask_RCNN)
devicenumber = 1
metric_cropsize = 1   # Larger values compute faster but are likely to provide lower numbers of detections.
                      # 1 m is an empirically proven good value for clast of pebble size.
kstart = 0
ksaveint = 25

#Detection & measurement operation
for i in range(0, numboffiles[0]):
    imgpath = os.path.join(UAVdirpath,filenames[i]) #path of each image to be analyzed
    clasts_list = clasts_detection.clasts_detect(mode = mode, 
                                                 metric_cropsize = metric_cropsize, 
                                                 imgpath = imgpath, 
                                                 plot = False, 
                                                 saveplot = False, 
                                                 saveresults = True, 
                                                 devicemode = devicemode, 
                                                 devicenumber = devicenumber, 
                                                 kstart = kstart, 
                                                 ksaveint = ksaveint)

# Detects and measures the clasts on a scaled image that can be either terrestrial (i.e. photograph of known resolution) or UAV derived (georeferenced ortho-image)
# mode: "terrestrial", "UAV"
# resolution: resolution of a terrestrial image (default = 0.001 m/pixel)
# imgpath: full path of the image/ortho-image
# metric_cropsize: Window size used for croping a UAV image into tiles (default = 1m)
# plot: (boolean) Display detections and histograms
# saveplot: (boolean) Save the detection and histogram figures
# saveresults: (boolean) Save the clast sizes into a CSV file
# devicemode: "gpu", "cpu" (default = "gpu") (see tesorflow-gpu documentation)
# devicenumber: id number of the device to be used (default = 0)
# kstart: Starting itteration step, (in case of resuming a previously stopped processing)(default = 0)
# ksaveint: Intervalle step between 2 checkpoint saving (default = 1%)

## Post-Processing Rasterization

In [ ]:
# Paths to the different files required for rasterization
ClastImageFilePath = os.path.join(UAVdirpath, 'example_n1_of_UAV_ortho_image.tif')
ClastFileName = 'example_n1_of_UAV_ortho_image_window_size=1m_individual_clast_values.csv'
ClastSizeListCSVFilePath = os.path.join(UAVdirpath, ClastFileName)

# Arguments list
field = 'Clast_length'
parameter='average'
cellsize=1
percentile=0.5
plot=False
figuresize = (15,20)
if parameter=='quantile':
    paramname='D'+str(int(percentile*100))
else:
    paramname=parameter
RasterFileWritingPath = os.path.join(UAVdirpath, ClastFileName[:-4]+'_field='+field+'_parameter='+paramname+'_cellsize='+str(cellsize)+'m.tif')

#Rasterization operation
clasts_raster = clasts_rasterize.clasts_rasterize(ClastImageFilePath, 
                                                  ClastSizeListCSVFilePath, 
                                                  RasterFileWritingPath, 
                                                  field, 
                                                  parameter, 
                                                  cellsize, 
                                                  percentile, 
                                                  plot, 
                                                  figuresize)

#     Converts the clast information from vector type to raster type.
#     ClastImageFilePath: Path of the geotiff file used to realize the clast detection and measurement
#     ClastSizeListCSVFilePath: Path of the CSV file containing the list of previously detected and measured clasts
#     RasterFileWritingPath: Path to be used for writing the raster produced by the present function
#     field: field (i.e. clast dimension) to be considered for computation (default = "Clast_length"):
#         - Clast_length
#         - Clast_width
#         - Ellipse_major_axis
#         - Ellipse_minor_axis
#         - Equivalent_diameter
#         - Score
#         - Orientation
#         - Surface_area
#         - Clast_elongation
#         - Ellipse_elongation
#         - Clast_circularity
#         - Ellipse_circularity
#         - Deposition_velocity: Current velocity (m/s) related to the clast deposition according to Hjulström's diagram #DOI: 10.1115/OMAE2013-10524
#         - Erosion_velocity: Current velocity (m/s) required in order to mobilize the clast according to Hjulström's diagram #DOI: 10.1115/OMAE2013-10524
#     parameter: Parameter to be computed for each cell: 
#         - "quantile": returns the quantile valued for the threshold specified by the "percentile" keyword 
#         - "density": returns the density of objects per cell size unit
#         - "average": returns the average value for each cell
#         - "std": returns the standard deviation for each cell
#         - "kurtosis": returns the kurtosis size for each cell
#         - "skewness": returns the skewness value for each cell
#     cellsize: Wanted output raster cell size (same unit as the geotiff file used to realize the clast detection and measurement
#     percentile: Percentile to be used for computing the quantile of each cell (default = 0.5, i.e. median)
#     plot: Switch for displaying the produced maps (default = True)
#     figuresize: Size of the displayed figure (default = (10,10))